# Custom DataLoader 문제

 - 기존 베이스라인 코드에서 모델을 MLP 대신에 pretrained 모델로 RoBERTa를 선택.
 - klue/roberta-large 모델을 선택하여 해당 모델에 맞게 전반적인 코드를 변경.
 - CustomDataset 부분에서 문제 발생.

---

The above exception was the direct cause of the following exception: KeyError

---

 - 위의 에러코드가 발생.
 - 에러코드에서 코드 흐름을 확인하니 tqdm -> torch/dataloader -> torch/fetch -> 내가 만든 CustomDataset 으로 이어졌다.
 - 클래스 내에서 에러가 발생한 부분은 'text = self.data['문장'][index]' 부분이다.
 - pandas/series에서 key_is_scalar: 항목으로 들어가 index에 맞게 레이블을 가져오는데
 - pandas/core/indexes/base.py in get_loc(self, key, method, tolerance) 항목에서 에러가 발생.

---

   3362             except KeyError as err:
-> 3363                 raise KeyError(key) from err

---

 - 해당 에러가 발생하고 그 아래 정수형(2 or 12726)으로 KeyError가 발생.


 - 인덱스 값이 전해지는 과정에서 에러가 생기는 것으로 보임.
 - loader가 아닌 dataset으로 내용물을 확인하면 2개는 텐서로 변환되어 반환되지만 3번째에서 동일한 에러가 발생.
 - CustomDataset을 만드는 과정에서 문제가 생긴것으로 보여 해당 항목 확인.

### 문제 해결

 - 아래 링크의 BERT를 사용한 베이스라인과 RoBERTa를 사용한 다른 대회 입상 코드를 확인하여 CustomDataset 제작을 확인.
 - https://dacon.io/competitions/official/236027/codeshare/6989?page=1&dtype=recent
 - https://dacon.io/competitions/official/236027/codeshare/7277?page=1&dtype=recent

---

 - 둘 다 데이터셋 과정에서 토크나이저를 불러와 사용하는데 베이스라인은 토크나이저 값을 그대로 반환하여 사용하고 입상 코드는 먼저 토크나이징을 하고 모델까지 불러와 집어넣어 반환한다.
 - 하지만 여전히 같은 방식으로 에러가 발생하였다.
 - 이 과정에서 발생한 에러는 기록하지는 못했지만 결국 토크나이저가 데이터셋을 만드는 과정 안에 있으면 에러가 나는 것으로 판단하였다.

---

 - 다른 방식들을 확인하는 과정에서 preprocessing 과정에서 먼저 데이터를 tensor로 변환하고서 데이터셋에 넣는 것을 보고서 해당 방식으로 시도.
 - id, mask를 토크나이저를 통해서 따로 만들고서 그것을 데이터셋에 input
 - 이후 train 과정까지 문제 없이 진행.